In [ ]:
# !pip install tf2onnx
# !pip install onnx
# !pip install onnxruntime
# !pip3 install numpy --upgrade

In [10]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import tf2onnx
import onnx
import cloudpickle
import numpy as np
import pandas as pd

In [5]:
model = tf.keras.models.load_model('models/review_sentiment_model.h5')
onnx_model, _ = tf2onnx.convert.from_keras(model)
onnx.save(onnx_model, "models/lstm_sentiment.onnx")

In [6]:
import onnxruntime
from tensorflow.keras.preprocessing.sequence import pad_sequences
from preprocessing.text_preprocessing import TextPreprocessor

In [7]:
with open("models/preprocessors.bin", 'rb') as f:
    tokenizer, maxlen, padding, truncating = cloudpickle.load(f)

In [11]:
session = onnxruntime.InferenceSession("models/lstm_sentiment.onnx", None)
def onnx_inference(text):
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    tp = TextPreprocessor()
    preprocessed_str = tp.preprocess(pd.Series(text), dataset="test")
    txt_seq = tokenizer.texts_to_sequences(preprocessed_str)
    input_ = {"embedding_input" :pad_sequences(txt_seq, maxlen=maxlen, padding=padding, truncating=truncating).astype(np.float32)}
    inputs = {key: value for key, value in input_.items()}
    result = session.run(None, inputs)[0][0]
    arg_max = np.argmax(result)  
    return [*result]

In [12]:
onnx_inference("it was very good")

[0.033652455, 0.9663476]

In [13]:
onnx_inference("it was very bad")

[0.99290097, 0.0070990026]

In [17]:
onnx_inference("it was very ugly")

[0.562265, 0.43773505]

In [19]:
onnx_inference("it was unhygenic")

[0.57017696, 0.429823]

In [20]:
onnx_inference("it was worst")

[0.99832803, 0.0016720059]